In [8]:
import torch

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

**BATCH_SIZE** 
딥러닝 모델에서 파라미터를 업데이트할 때 계산되는 데이터의 갯수, BATCH_SIZE만큼 데이터를 이용해 Output을 계산하고 BATCH_SIZE만큼 출력된 결괏값에 대한 오찻값을 계산, BATCH_SIZE만큼 계산된 오차값을 평균해 Back Propagation을 적용하고 이를 바탕으로 파라미터를 업데이트

**INPUT_SIZE** 
Input의 크기이자 입력층의 노드 수. 즉, INPUT_SIZE 크기의 벡터 값을 의미.
BATCH_SIZE가 64이고 INPUT_SIZE가 1000이란 말은 1000 크기의 벡터 값을 64개 이용한다는 의미 (64, 1000)

**HIDDEN_SIZE** 
Input을 다수의 파라미터를 이용해 계산한 결과에 한 번 더 계산되는 파라미터 수. 즉, 입력층에서 은닉층으로 전달됐을 때 은닉층의 노드 수.
아래 사이즈로 이야기하면 (64, 1000)의 Input들이 (1000, 100) 크기의 행렬과 행렬 곱을 계산하기 위해 설정한 수

**OUTPUT_SIZE** 
딥러닝 모델에서 최종으로 출력되는 값의 벡터 크기 
보통 Output의 크기는 최종으로 비교하고자 하는 레이블의 크기와 동일하게 설정 

In [9]:
BATCH_SIZE = 64 
INPUT_SIZE = 1000 
HIDDEN_SIZE = 100 
OUTPUT_SIZE = 10

In [12]:
# randn는 평균이 0, 표준편차가 1인 정규분포에서 샘플링한 값
# 크기가 2nd parameter 짜리의 벡터를 1st 파라미터개 만듬 
# input과 output에는 grad 설정할 필요X
x = torch.randn(BATCH_SIZE, INPUT_SIZE, device = DEVICE, dtype = torch.float, requires_grad = False)
y = torch.randn(BATCH_SIZE, OUTPUT_SIZE, device = DEVICE, dtype = torch.float, requires_grad = False)
w1 = torch.randn(INPUT_SIZE, HIDDEN_SIZE, device = DEVICE, dtype = torch.float, requires_grad = True)
w2 = torch.randn(HIDDEN_SIZE, OUTPUT_SIZE, device = DEVICE, dtype = torch.float, requires_grad = True)

In [13]:
learning_rate = 1e-6  # - 1
for t in range(1, 501):
    y_pred = x.mm(w1).clamp(min = 0).mm(w2)  # - 2
    
    loss = (y_pred - y).pow(2).sum()  # - 3
    if t % 100 == 0:
        print("Iteration: ", t, "\t", "Loss: ", loss.item())
    loss.backward()  # - 4
    
    with torch.no_grad():  # - 5
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        w1.grad.zero_()  # - 6
        w2.grad.zero_()

Iteration:  100 	 Loss:  514.2393798828125
Iteration:  200 	 Loss:  1.5027601718902588
Iteration:  300 	 Loss:  0.007492688484489918
Iteration:  400 	 Loss:  0.00018160293984692544
Iteration:  500 	 Loss:  3.211297371308319e-05


1 : 파라미터를 업데이트 시 Gradient를 계산한 결괏값에 learning_rate만큼 곱한 값을 이용해 업데이트, learning_rate를 어떻게 설정하느냐에 따라 Gradient 값에 따른 학습 정도가 결정, 딥러닝 모델에서 파라미터 값 업데이트시 가장 중요 하이퍼파라미터(HyperParameter)

2 : 딥러닝 모델의 결괏값을 보통 '예측값'이라 함. Input과 parameter 간의 행렬 곱을 이용한 후 clamp(비선형 함수)라는 메서드를 이용(활성화 함수), Clamp는 ReLU와 같은 역할을 함

3 : 예측값과 실제 레이블 값을 비교해 오차를 계산한 값을 loss라 함. pow 함수를 사용해 제곱차의 합을 sum을 이용해 계산

4 : 계산된 Loss 값에 대해 backward() 메서드를 이용해 각 파라미터 값에 대해 Gradient를 계산하고 Back Propagation(역전파법)을 진행.

5 : 각 파라미터 값에 대해 Gradient를 계산한 결과를 이용해 파라미터 값 업데이트시엔 해당 시점의 Gradient 값을 고정 후 업데이트를 진행 

6 : 각 파라미터 값을 업데이트 했다면 각 파라미터 값의 Gradient를 초기화해 다음 반복문을 진행할 수 있게 Gradient 값을 0으로 설정